In [1]:
# Load LSTM network and generate text
import sys
import numpy as np
#import mxnet as mx
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import CuDNNLSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.utils import multi_gpu_model

Using TensorFlow backend.
C:\Program Files\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# load ascii text and covert to lowercase
filename = "pride_and_prejudice.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [3]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [4]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  704175
Total Vocab:  58


In [5]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 30
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  704145


In [6]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [7]:
model = Sequential()
model.add(CuDNNLSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(CuDNNLSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [8]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [9]:
model.fit(X, y, epochs=20, batch_size=64, callbacks=callbacks_list)

Epoch 1/20
704145/704145 [==============================] - 173s 246us/step - loss: 2.5178

Epoch 00001: loss improved from inf to 2.51782, saving model to weights-improvement-01-2.5178.hdf5
Epoch 2/20
704145/704145 [==============================] - 169s 240us/step - loss: 2.0570

Epoch 00002: loss improved from 2.51782 to 2.05704, saving model to weights-improvement-02-2.0570.hdf5
Epoch 3/20
704145/704145 [==============================] - 164s 233us/step - loss: 1.8775

Epoch 00003: loss improved from 2.05704 to 1.87745, saving model to weights-improvement-03-1.8775.hdf5
Epoch 4/20
704145/704145 [==============================] - 162s 231us/step - loss: 1.7794

Epoch 00004: loss improved from 1.87745 to 1.77936, saving model to weights-improvement-04-1.7794.hdf5
Epoch 5/20
704145/704145 [==============================] - 165s 235us/step - loss: 1.7151

Epoch 00005: loss improved from 1.77936 to 1.71514, saving model to weights-improvement-05-1.7151.hdf5
Epoch 6/20
704145/704145 [===